
# Assignment 4 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


    This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `Corona_NLP_train.csv` that you can get from the course website [git](https://github.com/datascience-intro/1MS041-2024/blob/main/notebooks/data/Corona_NLP_train.csv). The data is "Coronavirus tweets NLP - Text Classification" that can be found on [kaggle](https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification). The data has several columns, but we will only be working with `OriginalTweet`and `Sentiment`.

1. [3p] Load the data and filter out those tweets that have `Sentiment`=`Neutral`. Let $X$ represent the `OriginalTweet` and let 
    $$
        Y = 
        \begin{cases}
        1 & \text{if sentiment is towards positive}
        \\
        0 & \text{if sentiment is towards negative}.
        \end{cases}
    $$
    Put the resulting arrays into the variables $X$ and $Y$. Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, this is to make sure that we all did the same splits (we are in this case assuming the data is IID as presented in the dataset). That is [train,test,validation] is the splitting layout.

2. [4p] There are many ways to solve this classification problem. The first main issue to resolve is to convert the $X$ variable to something that you can feed into a machine learning model. For instance, you can first use [`CountVectorizer`](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) as the first step. The step that comes after should be a `LogisticRegression` model, but for this to work you need to put together the `CountVectorizer` and the `LogisticRegression` model into a [`Pipeline`](https://scikit-learn.org/1.5/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline). Fill in the variable `model` such that it accepts the raw text as input and outputs a number $0$ or $1$, make sure that `model.predict_proba` works for this. **Hint: You might need to play with the parameters of LogisticRegression to get convergence, make sure that it doesn't take too long or the autograder might kill your code**
3. [3p] Use your trained model and calculate the precision and recall on both classes. Fill in the corresponding variables with the answer.
4. [3p] Let us now define a cost function
    * A positive tweet that is classified as negative will have a cost of 1
    * A negative tweet that is classified as positive will have a cost of 5
    * Correct classifications cost 0
    
    complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). 

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, fill in the selected threshold value in value `optimal_threshold`.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 99% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted (one of the outputs of `predict_proba`), if we define the random variable
    $$
        C = (1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
    $$
    then $C$ denotes the cost of a randomly chosen tweet. In the previous step we estimated $\mathbb{E}[C]$ using the empirical mean. However, since the threshold is chosen to minimize cost it is likely that $C=0$ or $C=1$ than $C=5$ as such it will have a low variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [2]:

# Part 1

# Load the data from the file specified in the problem definition and make sure that it is loaded using
# the search path `data/Corona_NLP_train.csv`. This is to make sure the autograder and your computer have the same
# file path and can load the data correctly.

# Contrary to how many other problems are structured, this problem actually requires you to
# have X on the shape (n_samples, ) that is a 1-dimensional array. Otherwise it will cause a bunch
# of errors in the autograder or also in for instance CountVectorizer.

# Make sure that all your data is numpy arrays and not pandas dataframes or series.

import csv
import numpy as np 
X = []
Y =[]
with open('data/Corona_NLP_train.csv', mode='r',encoding='ISO-8859-1') as csv_file:
    reader = csv.reader(csv_file)
    next(reader)
    for row in reader:
        if 'neutral' in row[5].lower():
            continue
        if 'positive' in row[5].lower():
            Y.append(1)
        else:
            Y.append(0)
        X.append(row[4].strip())

train_nr = int(len(X) * 0.6)
train_test_nr = int(len(X) * 0.75)
X_train =  np.array(X[:train_nr])
Y_train = np.array(Y[:train_nr])
X_test = np.array(X[train_nr:train_test_nr-1])
Y_test =np.array( Y[train_nr:train_test_nr-1])
X_valid = np.array(X[train_test_nr-1:])
Y_valid = np.array(Y[train_test_nr-1:])

In [3]:

# Part 2

# Train a machine learning model or pipeline that can take the raw strings from X and predict Y=0,1 depending on the
# sentiment of the tweet. Store the trained model in the variable `model`.
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import precision_score, recall_score

model  = Pipeline([('convetor', CountVectorizer()),('logistic', LogisticRegression(max_iter=100, solver='liblinear', random_state=42))])
model.fit(X_train, Y_train)
model_test_predictions = model.predict(X_test)
model__test_probabilities = model.predict_proba(X_test)[:,1]


In [4]:

# Part 3

# Evaluate the model on the test set and calculate precision, and recall on both classes. Store the results in the
# variables `precision_0`, `precision_1`, `recall_0`, `recall_1`.
precision_0 = precision_score(Y_test, model_test_predictions, pos_label=0)
precision_1 = precision_score(Y_test, model_test_predictions, pos_label=1)
recall_0 = recall_score(Y_test, model_test_predictions, pos_label=0)
recall_1 = recall_score(Y_test, model_test_predictions, pos_label=1)
print(precision_0,precision_1,recall_0,recall_1)



0.8579595426561126 0.8708971553610503 0.8464208242950109 0.8808557727775729


In [5]:

# Part 4

def cost(model,threshold,X,Y):
    # Hint, make sure that the model has a predict_proba method
    # think about how the decision is made based on the probabilities
    # and how the threshold can be used to make the decision.
    # For reference take a look at the lecture notes "Bayes classifier"
    # which contains how the decision is made based on the probabilities when the threshold is 0.5.
    
    # Fill in what is missing to compute the cost and return it
    # Note that we are interested in average cost
    model_probabilities = model.predict_proba(X)[:,1]
    model_adjusted_labels = (model_probabilities > threshold).astype(int)
    model_fn_cost = np.sum((model_adjusted_labels == 0) & (Y == 1))
    model_fp_cost = np.sum((model_adjusted_labels == 1) & (Y == 0)) * 5 
    total = model_fn_cost + model_fp_cost
    return total / len(Y)

In [6]:

# Part 5

# Find the optimal threshold for the model on the test set. Store the threshold in the variable `optimal_threshold`
# and the cost at the optimal threshold in the variable `cost_at_optimal_threshold` evaluated on the test set.

min_cost = -1
min_threshold = 0
potential_thresholds = np.arange(0, 1.01, 0.1)
for threshold in potential_thresholds:
    computed_cost = cost(model, threshold, X_test, Y_test)
    if min_cost == -1:
        min_cost = computed_cost
        min_threshold = threshold
    elif min_cost > computed_cost:
        min_cost = computed_cost
        min_threshold = threshold
optimal_threshold = min_threshold
cost_at_optimal_threshold = min_cost
print(optimal_threshold, cost_at_optimal_threshold)

0.8 0.2727272727272727


In [7]:

# Part 6
cost_at_optimal_threshold_valid = cost(model,optimal_threshold,X_valid,Y_valid)
print(cost_at_optimal_threshold_valid)
e_hoeffdings = np.sqrt(25 * np.log(2 / 0.01) / 2 / len(Y_valid))
print(e_hoeffdings)
cost_interval_valid = (cost_at_optimal_threshold_valid - e_hoeffdings, cost_at_optimal_threshold_valid + e_hoeffdings)
print(cost_interval_valid)

assert(type(cost_interval_valid) == tuple)
assert(len(cost_interval_valid) == 2)

0.27744558718010043
0.08899567661998982
(np.float64(0.18844991056011062), np.float64(0.36644126380009023))


In [8]:
# Part 7
from scipy.optimize import root_scalar

probabilities_validation = model.predict_proba(X_valid)[:, 1]
adjusted_labels_validation = (probabilities_validation >= optimal_threshold).astype(int)
validation_value_of_C = (1 - adjusted_labels_validation) * Y_valid + 5 * (1 - Y_valid) * adjusted_labels_validation


variance_of_C = np.var(validation_value_of_C)
print(variance_of_C)

def determine_epsilon(e):
    x = (1 + (5 * e) / variance_of_C) * np.log(1 + (5 * e) / variance_of_C) - (5 * e) / variance_of_C
    return 2 * np.exp(-len(validation_value_of_C) * (variance_of_C / (5 ** 2)) * x) - 0.01  

e_bennett = root_scalar(determine_epsilon, bracket=[0, 0.2], method="bisect").root

mean_of_C = np.mean(validation_value_of_C)
print(mean_of_C)
interval_of_C = (mean_of_C - e_bennett, mean_of_C + e_bennett)
print(interval_of_C)

assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)

0.6692569047766288
0.27744558718010043
(np.float64(0.24728539492103696), np.float64(0.3076057794391639))
